In [1]:
from datetime import datetime, date
import os
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode, quote
import json

In [2]:
download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # по этому адресу будет качать
get_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?' # а с этого получать имена файлов
folder_url = 'https://disk.yandex.ru/d/xKJ4KlXJIJEULw'
#url = base_url + urlencode(dict(public_key=folder_url, path=file_url), safe = '/', quote_via=quote)
url_get = get_url + urlencode(dict(public_key=folder_url)) 

In [3]:
r_get = requests.get(url_get)  # получаем соединение. У него есть параметр text, с помощью него будем получать имена файлов
print(r_get)
#print(r_get.text)

<Response [200]>


In [4]:
#json.loads(r_get.text)  # вот словарь, полученный с json. Нам нужен ключ _embedded, и в нем - ключ items

In [5]:
files = json.loads(r_get.text)['_embedded']['items']
#files  ## и здесь можно брать ключ name, в котором имя файла

In [6]:
Cbis_df = None

In [7]:
for file in files:
    if file['name'].endswith('.csv'):
        file_url = file['path']
        params = {
            'public_key':folder_url,
            'path':file_url
        }
        url = download_url + urlencode(params)
        r = requests.get(url) 
        download_link = json.loads(r.text)['href']
        if Cbis_df is None:
            Cbis_df = pd.read_csv(download_link, sep = ';', encoding='1251')
        else:
            dop_df = pd.read_csv(download_link, sep = ';', encoding='1251')
            Cbis_df = pd.concat([Cbis_df, dop_df], ignore_index=True)


In [8]:
Cbis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295737 entries, 0 to 295736
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Дата                   295737 non-null  object 
 1   Номер                  295737 non-null  object 
 2   Сумма                  295737 non-null  object 
 3   Статус                 295737 non-null  object 
 4   Примечание             0 non-null       float64
 5   Комментарий            134903 non-null  object 
 6   Контрагент             295737 non-null  object 
 7   ИНН/КПП                295737 non-null  object 
 8   Организация            295737 non-null  object 
 9   ИНН/КПП.1              295737 non-null  object 
 10  Тип документа          290231 non-null  object 
 11  Имя файла              295737 non-null  object 
 12  Дата.1                 206370 non-null  object 
 13  Номер.1                152476 non-null  object 
 14  Сумма.1                295737 non-nu

In [9]:
Cbis_df.head()

,Дата,Номер,Сумма,Статус,Примечание,Комментарий,Контрагент,ИНН/КПП,Организация,ИНН/КПП.1,...,Время,Тип пакета,Идентификатор пакета,Запущено в обработку,Получено контрагентом,Завершено,Увеличение суммы,НДС,Уменьшение суммы,НДС.1
0,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,30.09.21 23:40,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
1,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"
2,30.09.21,БРН00709545,"0,00",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"
3,30.09.21,БРН00709520,"54 705,65",Выполнение завершено успешно,NaN,"БРН00709520 на сумму 54 705,65р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:34:51,ДокОтгрВх,1cfdcbd4-a43b-4543-90b6-59d0e6095b94,30.09.21 23:34,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
4,30.09.21,БРН00709520,"54 705,65",Выполнение завершено успешно,NaN,"БРН00709520 на сумму 54 705,65р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:34:51,ДокОтгрВх,1cfdcbd4-a43b-4543-90b6-59d0e6095b94,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"


In [10]:
Cbis_df.columns = Cbis_df.columns.str.replace(' ', '_')

In [11]:
Cbis_df.columns

Index(['Дата', 'Номер', 'Сумма', 'Статус', 'Примечание', 'Комментарий',
       'Контрагент', 'ИНН/КПП', 'Организация', 'ИНН/КПП.1', 'Тип_документа',
       'Имя_файла', 'Дата.1', 'Номер.1', 'Сумма.1', 'Сумма_НДС',
       'Ответственный', 'Подразделение', 'Код', 'Дата.2', 'Время',
       'Тип_пакета', 'Идентификатор_пакета', 'Запущено_в_обработку',
       'Получено_контрагентом', 'Завершено', 'Увеличение_суммы', 'НДС',
       'Уменьшение_суммы', 'НДС.1'],
      dtype='object')

In [12]:
float_columns = ['Сумма', 'Сумма.1', 'Сумма_НДС', 'НДС', 'Уменьшение_суммы', 'НДС.1']

In [13]:
for column in float_columns:
    Cbis_df[column] = Cbis_df[column].str.replace(' ', '').str.replace(',', '.').astype('float')

### Получение данных с аптек

In [14]:
download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
get_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'
apteks_url = 'https://disk.yandex.ru/d/m9MtFWt56fDW7g'
path_to_folder = '/csv/correct'

In [15]:
apteka_url_get = get_url + urlencode({'public_key':apteks_url, 'path':path_to_folder}, quote_via = quote)
apteka_url_get

'https://cloud-api.yandex.net/v1/disk/public/resources?public_key=https%3A%2F%2Fdisk.yandex.ru%2Fd%2Fm9MtFWt56fDW7g&path=%2Fcsv%2Fcorrect'

In [16]:
apt_r_get = requests.get(apteka_url_get)
apt_files = json.loads(apt_r_get.text)['_embedded']['items']

In [17]:
need_doc_type = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

In [18]:
filtered_cbis = Cbis_df[Cbis_df['Тип_документа'].isin(need_doc_type)]

In [19]:
file_path_pattern = f'Результат/{date.today()}/'
file_path_pattern

'Результат/2024-11-05/'

In [20]:
if not os.path.exists(file_path_pattern):
    os.makedirs(file_path_pattern)

In [21]:
df_columns = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
'Дата приходного документа', 'Номер приходного документа',
'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
'Сумма счет-фактуры', 'Кол-во',
'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

In [23]:
for file in apt_files:
    if file['name'].endswith('.csv'):
        file_url = file['path']
        params = {
            'public_key':apteks_url,
            'path': file_url
        }
        url = download_url + urlencode(params)
        r = requests.get(url) 
        download_link = json.loads(r.text)['href']
        df = pd.read_csv(download_link, sep = ';', encoding='1251')
        
        df['Номер накладной'] = df['Номер накладной'].astype(str)
        df["Номер счет-фактуры"] = ''
        df['Сумма счет-фактуры'] = pd.Series(dtype = 'float')
        df['Дата счет-фактуры'] = ''
        df['Сравнение дат'] = ''
        
        for i, row in df.iterrows():
            waybill = row['Номер накладной']
            if 'ЕАПТЕКА' in row['Поставщик']:
                waybill +='/15'
            records = filtered_cbis[filtered_cbis['Номер'] == waybill]
            if records.empty:
                continue
            numb, summ, dt = records[['Номер', 'Сумма', 'Дата']].iloc[0]
            dt = datetime.strptime(dt, '%d.%m.%y').strftime('%d.%m.%Y')
            df.at[i, "Номер счет-фактуры"] = numb
            df.at[i, 'Сумма счет-фактуры'] = summ
            df.at[i, 'Дата счет-фактуры'] = dt
            if df.at[i, 'Дата накладной'] != dt:
                df.at[i, 'Сравнение дат']= 'Не совпадает!'
            
        df = df[df_columns]
        path = file_path_pattern + file['name'][:-4] + ' - результат.xlsx'
        df.to_excel(path, index = False, float_format="%.2f")
        print(f'{file["name"]} Обработан!')


366.csv Обработан!
А123.csv Обработан!
